In [ ]:
import requests
from lxml import etree
from time import sleep
import pandas as pd

In [ ]:
STOP = False
PAGE_NUM = 1
ADDRESS = '0x99dc81489b75268baad66e2eb2301371d1ce235a'
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
}
data_to_df = []

while(not STOP):
    res = requests.get(f'https://etherscan.io/txs?a={ADDRESS}&p={PAGE_NUM}',headers=headers)
    raw_data = etree.HTML(res.text)
    total_page = int(raw_data.xpath("//div[@id='ContentPlaceHolder1_topPageDiv']//strong[@class='font-weight-medium'][2]/text()")[-1])
    total_rows = len(raw_data.xpath("//tbody/tr"))+1
    
    print(f"Current page {PAGE_NUM}/{total_page}")
    
    for row_num in range(1,total_rows):
        temp = {
            'txn' : raw_data.xpath(f"//tbody/tr[{row_num}]/td[2]/span/a/text()")[-1],
            'method' : raw_data.xpath(f"//tbody/tr[{row_num}]/td[3]/span/text()")[-1],
            'block' : raw_data.xpath(f"//tbody/tr[{row_num}]/td[4]/a/text()")[-1],
            'age' : raw_data.xpath(f"//tbody/tr[{row_num}]/td[6]/span/text()")[-1],
            'value' : raw_data.xpath(f"//tbody/tr[{row_num}]/td[10]/text()")[-1],
            'txn_fee' : '.'.join(raw_data.xpath(f"//tbody/tr[{row_num}]/td[11]/span/text()"))
        }
        
        #########################
        ## FROM ##
        if raw_data.xpath(f"//tbody/tr[{row_num}]/td[7]/span/a"):
            temp['from'] = raw_data.xpath(f"//tbody/tr[{row_num}]/td[7]/span/a/text()")[-1]
        elif raw_data.xpath(f"//tbody/tr[{row_num}]/td[7]/a"):
            temp['from'] = raw_data.xpath(f"//tbody/tr[{row_num}]/td[7]/a/text()")[-1]
        elif raw_data.xpath(f"//tbody/tr[{row_num}]/td[7]/span"):
            temp['from'] = raw_data.xpath(f"//tbody/tr[{row_num}]/td[7]/span/text()")[-1]
        else:
            temp['from'] = None

        ## TO ##
        if raw_data.xpath(f"//tbody/tr[{row_num}]/td[9]/span/span"):
            temp['to'] = raw_data.xpath(f"//tbody/tr[{row_num}]/td[9]/span/span")[-1].attrib['title']
        elif raw_data.xpath(f"//tbody/tr[{row_num}]/td[9]/span/span/a/text()"):
            temp['to'] = raw_data.xpath(f"//tbody/tr[{row_num}]/td[9]/span/span/text()")[-1]
        elif raw_data.xpath(f"//tbody/tr[{row_num}]/td[9]/span/a/text()"):
            temp['to'] = raw_data.xpath(f"//tbody/tr[{row_num}]/td[9]/span/a/text()")[-1]
        elif raw_data.xpath(f"//tbody/tr[{row_num}]/td[9]/a/text()"):
            temp['to'] = raw_data.xpath(f"//tbody/tr[{row_num}]/td[9]/a/text()")[-1]
        else:
            temp['to'] = None
        #########################
        
        data_to_df.append(temp)
    
    if PAGE_NUM >= total_page:
        STOP = True

    PAGE_NUM+=1
    sleep(1)

In [ ]:
data_to_df

In [ ]:
columns = ['txn','method','block','age','value','txn_fee','from','to']
df = pd.DataFrame(data_to_df,columns=columns)
df.head()

In [ ]:
df.to_csv("this_is_web_scrapy.csv",encoding='utf-8')